In [2]:
import json
import os
import time

import pandas as pd
from tqdm import tqdm

# Простой Baseline VW

### Вариант с модификацией dataset

In [6]:
%%time

!vw -d ..\\vw\\dataset.vw --loss_function logistic -f ..\\models\\logistic.model --passes 3 --ngram 2 --holdout_off --cache_file ..\\models\\cache.cache --random_seed 17

Generating 2-grams for all namespaces.
final_regressor = ..\\models\\logistic.model
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = ..\\models\\temp.cache
Reading datafile = ..\\vw\\dataset.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.693147 0.693147            1            1.0  -1.0000   0.0000       48
0.491887 0.290626            2            2.0  -1.0000  -1.0869       46
0.319663 0.147439            4            4.0  -1.0000  -1.5767       46
0.217190 0.114716            8            8.0  -1.0000  -1.8737       35
0.145423 0.073657           16           16.0  -1.0000  -3.6736      138
0.093701 0.041980           32           32.0  -1.0000  -3.1650       64
0.057883 0.022065           64           64.0  -1.0000  -3.8537       37
0.035184 0.012486          128          128.0  -1.0000  -4.3918    

Wall time: 7min 41s


In [7]:
%%time

# Предсказание на трейне
!vw -i ..\\models\\logistic.model -t -d ..\\vw\\dataset_test.vw -p ..\\output\\pred_train_logistic_p3n2.txt --quiet

# Оптимизация параметров: wv-hyperopt.py

Оригинальный скрипт wv-hyperopt.py доступен на github: https://github.com/JohnLangford/vowpal_wabbit/tree/master/utl

Обучение модели на stratified shuffled данных 90% от подготовленного датасета. Валидация на 10% shuffled.

In [1]:
# Соотношение target и величины выборки

print((21452 + 406542) / 406542)
print((21452 + 406542) / 21452)

1.0527669957839534
19.951239977624464


In [ ]:
# Команда для оптимизации параметров (запуск из командной строки)
python vw-hyperopt.py --train "..\\vw\\dataset_train_shuffled.vw" --holdout "..\\vw\\dataset_valid_shuffled.vw" --max_evals 100 --outer_loss_function roc-auc --vw_space "--algorithms=ftrl,sgd -b=28 --ngram=c2,c3,c4 --ftrl_alpha=5e-5..8e-1~L --ftrl_beta=0.01..1 --classweight=-1:0.02,-1:0.05,-1:0.1,-1:0.2,-1:0.5~O --l2=1e-8..1e-2~LO --l1=1e-8..1e-2~LO -l=0.01..1~L --decay_learning_rate=0.95,0.97~O --power_t=0.01..1 --passes=3..10~I --loss_function=logistic" --plot

## Загрузить результаты оптимизации

In [6]:
with open('dataset_train_shuffled_trials.json') as f:
    params = json.load(f)

params_df = pd.DataFrame(params).sort_values(by=['loss'], axis=0).reset_index(drop=True)
params_df.head(10)

,current_trial,eval_time,loss,status,train_command
0,33,266,-0.588495,ok,vw -d ..\\vw\\dataset_train_shuffled.vw -f ./d...
1,42,323,-0.588484,ok,vw -d ..\\vw\\dataset_train_shuffled.vw -f ./d...
2,34,296,-0.588474,ok,vw -d ..\\vw\\dataset_train_shuffled.vw -f ./d...
3,35,327,-0.588457,ok,vw -d ..\\vw\\dataset_train_shuffled.vw -f ./d...
4,74,295,-0.588444,ok,vw -d ..\\vw\\dataset_train_shuffled.vw -f ./d...
5,71,325,-0.588409,ok,vw -d ..\\vw\\dataset_train_shuffled.vw -f ./d...
6,68,315,-0.588381,ok,vw -d ..\\vw\\dataset_train_shuffled.vw -f ./d...
7,66,304,-0.588282,ok,vw -d ..\\vw\\dataset_train_shuffled.vw -f ./d...
8,30,234,-0.588228,ok,vw -d ..\\vw\\dataset_train_shuffled.vw -f ./d...
9,31,262,-0.588152,ok,vw -d ..\\vw\\dataset_train_shuffled.vw -f ./d...


In [3]:
for row, command in params_df.iterrows():
    print(row, ':', command['loss'], "\n", command['train_command'], '\n')

0 : -0.588495297878427 
 vw -d ..\\vw\\dataset_train_shuffled.vw -f ./dataset_train_shuffled_current.model --holdout_off -c  --decay_learning_rate 0.97 --loss_function logistic --ngram c3 --passes 8 --power_t 0.21846519028073763 -b 28 -l 0.02203702669188238  

1 : -0.5884835125943594 
 vw -d ..\\vw\\dataset_train_shuffled.vw -f ./dataset_train_shuffled_current.model --holdout_off -c  --decay_learning_rate 0.95 --loss_function logistic --ngram c3 --passes 10 --power_t 0.24095213922984413 -b 28 -l 0.021387359069161088  

2 : -0.5884736863324582 
 vw -d ..\\vw\\dataset_train_shuffled.vw -f ./dataset_train_shuffled_current.model --holdout_off -c  --decay_learning_rate 0.97 --loss_function logistic --ngram c3 --passes 9 --power_t 0.20476209324788347 -b 28 -l 0.018642027897230323  

3 : -0.5884571256956075 
 vw -d ..\\vw\\dataset_train_shuffled.vw -f ./dataset_train_shuffled_current.model --holdout_off -c  --decay_learning_rate 0.97 --loss_function logistic --ngram c3 --passes 10 --power_t 0

 vw -d ..\\vw\\dataset_train_shuffled.vw -f ./dataset_train_shuffled_current.model --holdout_off -c  --decay_learning_rate 0.95 --ftrl_alpha 0.05317310857441863 --ftrl_beta 0.7530902424172148 --l1 0.00044735977325157365 --l2 0.00018679054532353537 --loss_function logistic --ngram c4 --passes 6 --power_t 0.43719027984742387 -b 28 -l 0.04923834162529151 --ftrl 

59 : -0.5590694546059615 
 vw -d ..\\vw\\dataset_train_shuffled.vw -f ./dataset_train_shuffled_current.model --holdout_off -c  --decay_learning_rate 0.97 --loss_function logistic --ngram c3 --passes 8 --power_t 0.15923635009010725 -b 28 -l 0.7485630313065545  

60 : -0.5590441804498597 
 vw -d ..\\vw\\dataset_train_shuffled.vw -f ./dataset_train_shuffled_current.model --holdout_off -c  --classweight -1:0.1 --decay_learning_rate 0.97 --ftrl_alpha 0.08318320418603552 --ftrl_beta 0.28460336913806034 --l1 3.72632294925409e-07 --l2 2.087464513560191e-06 --loss_function logistic --ngram c3 --passes 4 --power_t 0.7925417556353701 -b 28 

## Модели с оптимизированными параметрами

Выбрать некоторые модели, обучить и сохранить результаты предсказаний

In [6]:
# Выбранные модели для обучения
# models_n = list(range(15,25))
models_n = [1, 14, 15, 20, 21, 27, 44, 46, 56, 57]

data_source = '../vw/dataset_shuffled.vw'

for command, num in zip(params_df['train_command'][models_n], models_n):
    init_time = time.time()
    
    model_name = '../models/dataset_shuffled_h' + str(num) + '.model'
    
    # --------------
    # Обучить модель
    # --------------
    vw_call_string = ' '.join(['vw', '-d', data_source, '-f', model_name, command.split('model ')[1]])
    
    print(f'Learning command ({num}/{len(models_n)}): {vw_call_string}')
    res = os.system(vw_call_string)
    
    print(f'Learning success. Elapsed: {round(time.time() - init_time, 2)} sec.' 
          if not res else 'Failed.')
    
    # ------------
    # Предсказания
    # ------------
    time.sleep(2)
    init_time = time.time()
    
    # Название файла с предсказаниями
    pred_file = '../output/h_' + str(num) + '.txt'
    
    vw_pred_string = f'vw -i {model_name} -t -d ../vw/dataset_test.vw -p {pred_file} --quiet'
    print(f'Predicting command ({num}/{len(models_n)}): {vw_pred_string}')
    res = os.system(vw_pred_string)
    
    print(f'Predicting success. Wrote predictions to {pred_file}. Elapsed: {round(time.time() - init_time, 2)} sec.'
          if not res else 'Failed.')


Learning command (1/10): vw -d ../vw/dataset_shuffled.vw -f ../models/dataset_shuffled_h1.model --holdout_off -c  --decay_learning_rate 0.95 --loss_function logistic --ngram c3 --passes 10 --power_t 0.24095213922984413 -b 28 -l 0.021387359069161088 
Learning success. Elapsed: 430.04 sec.
Predicting command (1/10): vw -i ../models/dataset_shuffled_h1.model -t -d ../vw/dataset_test.vw -p ../output/h_1.txt --quiet
Predicting success. Wrote predictions to ../output/h_1.txt. Elapsed: 59.19 sec.
Learning command (14/10): vw -d ../vw/dataset_shuffled.vw -f ../models/dataset_shuffled_h14.model --holdout_off -c  --classweight -1:0.2 --decay_learning_rate 0.95 --loss_function logistic --ngram c3 --passes 10 --power_t 0.05652234736937908 -b 28 -l 0.012527003024277061 
Learning success. Elapsed: 1020.83 sec.
Predicting command (14/10): vw -i ../models/dataset_shuffled_h14.model -t -d ../vw/dataset_test.vw -p ../output/h_14.txt --quiet
Predicting success. Wrote predictions to ../output/h_14.txt. El

# Оценки вероятности и submissions

##### Загрузить все предсказания в один датафрейм

In [3]:
models_n = [1, 14, 15, 20, 21, 27, 44, 46, 56, 57]

# Загрузить начальный df
file_pred = '..\\output\\' + 'h_' + str(models_n[0]) + '.txt'
pred_df = pd.read_csv(file_pred, header=None, names=['day', 'cat_feature', 'cuid'], index_col=None, sep=' |_', engine='python', usecols=[1,2,3])

for nmodel in models_n:
    file_pred = '..\\output\\' + 'h_' + str(nmodel) + '.txt'
    temp_df = pd.read_csv(file_pred, header=None, names=['pred' + str(nmodel)], index_col=None, sep=' ', usecols=[0])
    pred_df = pd.concat([pred_df, temp_df], axis=1)

In [4]:
pred_df['ave'] = pred_df[['pred'+str(x) for x in models_n]].mean(axis=1)
pred_df.index = pred_df['cuid']
pred_df.head(2)

,day,cat_feature,cuid,pred1,pred14,pred15,pred20,pred21,pred27,pred44,pred46,pred56,pred57,ave
cuid,,,,,,,,,,,,,,
000014fe918d1f97a632a796f4948be8,day7,2,000014fe918d1f97a632a796f4948be8,-2.796509,-1.196943,-2.777098,0.281049,-2.841886,1.158517,-2.485964,-1.887757,-2.337493,-3.302210,-1.818629
000014fe918d1f97a632a796f4948be8,day8,2,000014fe918d1f97a632a796f4948be8,-3.207107,-1.606038,-3.157062,-0.528776,-3.178761,0.700543,-3.915512,-3.713463,-2.282472,-5.892112,-2.678076


##### Усредненный submission

In [28]:
# Последовательность cuids для теста
cuid_test = pd.read_csv('..\\input\\mlboot_test.tsv', index_col=[0])

submission = pd.DataFrame(columns=['ave'], index=cuid_test.index)

for cuid in tqdm(submission.index):
    submission['ave'][cuid] = out_df['ave'][cuid].mean()

submission.to_csv('..\\output\\baseline3_h_total_ave' + str(models_n) + '.txt', header=None, index=None)

submission.head()

100%|█████████████████████████████████| 181024/181024 [03:11<00:00, 945.82it/s]


,ave
cuid,
888b238b4d14c03173baa375a739f6bc,-2.17446
ac4b8244f3ae82df511b002257473c11,-2.16608
483d8b91e49522c8a5bbe37f3872c749,-1.72051
4c7ec46a0e88a7e1e1cedd2d526d5d61,-2.28438
fdbfba9842ff0bf86d600eb334c7c42b,-2.19152


# Предсказания vw на dataset

In [7]:
models_n = [1, 14, 15, 20, 21, 27, 44, 46, 56, 57]

for command, num in zip(params_df['train_command'][models_n], models_n):
    # ------------
    # Предсказания
    # ------------
    init_time = time.time()
    
    model_name = '../models/dataset_shuffled_h' + str(num) + '.model'
    
    # Название файла с предсказаниями
    pred_file = '../output/dataset_h_' + str(num) + '.txt'
    
    vw_pred_string = f'vw -i {model_name} -t -d ../vw/dataset.vw -p {pred_file} --quiet'
    print(f'Predicting command ({num}/{len(models_n)}): {vw_pred_string}')
    res = os.system(vw_pred_string)
    
    print(f'Predicting success. Wrote predictions to {pred_file}. Elapsed: {round(time.time() - init_time, 2)} sec.'
          if not res else 'Failed.')

Predicting command (1/10): vw -i ../models/dataset_shuffled_h1.model -t -d ../vw/dataset.vw -p ../output/dataset_h_1.txt --quiet
Predicting success. Wrote predictions to ../output/dataset_h_1.txt. Elapsed: 122.6 sec.
Predicting command (14/10): vw -i ../models/dataset_shuffled_h14.model -t -d ../vw/dataset.vw -p ../output/dataset_h_14.txt --quiet
Predicting success. Wrote predictions to ../output/dataset_h_14.txt. Elapsed: 92.44 sec.
Predicting command (15/10): vw -i ../models/dataset_shuffled_h15.model -t -d ../vw/dataset.vw -p ../output/dataset_h_15.txt --quiet
Predicting success. Wrote predictions to ../output/dataset_h_15.txt. Elapsed: 92.72 sec.
Predicting command (20/10): vw -i ../models/dataset_shuffled_h20.model -t -d ../vw/dataset.vw -p ../output/dataset_h_20.txt --quiet
Predicting success. Wrote predictions to ../output/dataset_h_20.txt. Elapsed: 91.81 sec.
Predicting command (21/10): vw -i ../models/dataset_shuffled_h21.model -t -d ../vw/dataset.vw -p ../output/dataset_h_21.

In [8]:
models_n = [1, 14, 15, 20, 21, 27, 44, 46, 56, 57]

# Загрузить начальный df
file_pred = '..\\output\\dataset_' + 'h_' + str(models_n[0]) + '.txt'
pred_df = pd.read_csv(file_pred, header=None, names=['day', 'cat_feature', 'cuid'], index_col=None, sep=' |_', engine='python', usecols=[1,2,3])

for nmodel in models_n:
    file_pred = '..\\output\\' + 'h_' + str(nmodel) + '.txt'
    temp_df = pd.read_csv(file_pred, header=None, names=['pred' + str(nmodel)], index_col=None, sep=' ', usecols=[0])
    pred_df = pd.concat([pred_df, temp_df], axis=1)

##### Дополнительные признаки

In [9]:
# Среднее значение
pred_df['ave'] = pred_df[['pred'+str(x) for x in models_n]].mean(axis=1)
pred_df.index = pred_df['cuid']
pred_df.head(2)

,day,cat_feature,cuid,pred1,pred14,pred15,pred20,pred21,pred27,pred44,pred46,pred56,pred57,ave
cuid,,,,,,,,,,,,,,
00000d2994b6df9239901389031acaac,day39,5,00000d2994b6df9239901389031acaac,-2.796509,-1.196943,-2.777098,0.281049,-2.841886,1.158517,-2.485964,-1.887757,-2.337493,-3.302210,-1.818629
00000d2994b6df9239901389031acaac,day43,5,00000d2994b6df9239901389031acaac,-3.207107,-1.606038,-3.157062,-0.528776,-3.178761,0.700543,-3.915512,-3.713463,-2.282472,-5.892112,-2.678076


In [38]:
# Признак день

days = pred_df['day'].unique().tolist()
days = [f'day{int(day[3:]):02d}' for day in days]
days.sort()

days_df = pd.DataFrame(0, index=pred_df.index, columns=days)

### Загрузить данные

In [58]:
file_pred = '..\\output\\h_20.txt'
pred_df = pd.read_csv(file_pred, header=None, names=['pred', 'day', 'cat_feature', 'cuid'], index_col=None, sep=' |_', engine='python')
pred_df.head()

,pred,day,cat_feature,cuid
0,0.281049,day7,2,000014fe918d1f97a632a796f4948be8
1,-0.528776,day8,2,000014fe918d1f97a632a796f4948be8
2,0.061917,day11,2,000014fe918d1f97a632a796f4948be8
3,0.203633,day13,2,000014fe918d1f97a632a796f4948be8
4,0.170326,day14,2,000014fe918d1f97a632a796f4948be8


In [59]:
file_cuids = '..\\vw\\dataset_cuid_test.vw'
file_test = '..\\input\\mlboot_test.tsv'

cuid_df = pd.read_csv(file_cuids, index_col=None, names=['cuid'], usecols=[0], header=None, sep='\t')
cuid_test = pd.read_csv(file_test, index_col=[0])

pred_df.shape, cuid_df.shape

((6654252, 4), (6654252, 1))

### Использование мультииндекса
#### Создание датафрейма

In [86]:
multi_index = pd.MultiIndex.from_tuples([(x, y) for x, y in zip(cuid_df['cuid'], pred_df['day'])])

out_df_multi = pred_df
out_df_multi.index = multi_index

# Убрать дубликаты в индексе уровня 1
out_df_multi = out_df_multi[~out_df_multi.index.duplicated(keep='first')]
print('До/после удаления дубликатов в мультииндексе:', pred_df.shape, out_df_multi.shape)

# Развернуть датафрейм в столбцы
out_df_multi = out_df_multi['pred'].unstack(level=1)

# Отсортировать индексы как задано в test
out_df_multi = out_df_multi.reindex(cuid_out.index)

# Переименовать столбцы и упорядочить
names = dict([(f'day{x}', f'day{x:02d}') for x in range(1, 10)])
out_df_multi = out_df_multi.rename(columns=names)
out_df_multi = out_df_multi.sort_index(axis=1)

out_df_multi.head(3)

До/после удаления дубликатов в мультииндексе: (6654252, 2) (5542237, 2)


,day01,day02,day03,day04,day05,day06,day07,day08,day09,day10,...,day51,day52,day53,day54,day55,day56,day57,day58,day59,day60
cuid,,,,,,,,,,,,,,,,,,,,,
888b238b4d14c03173baa375a739f6bc,-5.006658,NaN,-2.521204,NaN,-2.771118,-2.221654,-2.577213,-2.700551,-2.849910,-2.786111,...,-3.347989,-2.716497,NaN,-3.245458,-2.544358,-2.828236,-2.735087,NaN,NaN,NaN
ac4b8244f3ae82df511b002257473c11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
483d8b91e49522c8a5bbe37f3872c749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.768052,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Усредненный submission

In [71]:
# Усредненный ответ
out_df_multi.mean(axis=1).to_csv('..\\output\\baseline2_h_1.txt', header=None, index=None)

### Усреднение предсказания без мультииндекса (без удаления дубликатов дней)

In [60]:
out_df = pred_df.copy()
out_df.index = cuid_df['cuid']

submission = pd.DataFrame(columns=['ave'], index=cuid_test.index)

for cuid in tqdm(submission.index):
    submission['ave'][cuid] = out_df['pred'][cuid].mean()

submission.to_csv('..\\output\\baseline3_h_20_ave.txt', header=None, index=None)

submission.head()

100%|████████████████████████████████| 181024/181024 [02:59<00:00, 1009.33it/s]


,ave
cuid,
888b238b4d14c03173baa375a739f6bc,-0.188752
ac4b8244f3ae82df511b002257473c11,-0.140454
483d8b91e49522c8a5bbe37f3872c749,0.113673
4c7ec46a0e88a7e1e1cedd2d526d5d61,-0.244325
fdbfba9842ff0bf86d600eb334c7c42b,-0.0791256


# Организация нового датасета из предсказаний test

In [21]:
pred_file = '..\\output\\pred_train_logistic_p3n2.txt'
data_cuids = '..\\vw\\dataset_cuid.vw'
data = '..\\input\\mlboot_test.tsv'

input_df = pd.read_csv(pred_file, header=None, names=['pred', 'day'], index_col=None, sep=' ')
cuid_df = pd.read_csv(data_cuids, index_col=None, names=['cuid'], usecols=[0], header=None, sep='\t')
cuid_out = pd.read_csv(data, index_col=[0])

input_df.shape, cuid_df.shape

((12874345, 2), (12874345, 1))

In [23]:
input_df['cuid'] = cuid_df['cuid']
input_df

,pred,day,cuid
0,-3.148156,day39,00000d2994b6df9239901389031acaac
1,-3.447425,day43,00000d2994b6df9239901389031acaac
2,-3.372123,day45,00000d2994b6df9239901389031acaac
3,-3.472649,day46,00000d2994b6df9239901389031acaac
4,-4.678955,day48,00000d2994b6df9239901389031acaac
5,-3.520002,day53,00000d2994b6df9239901389031acaac
6,-2.640258,day55,00000d2994b6df9239901389031acaac
7,-3.146274,day14,00000d53c06c43379ed22c775ce89614
8,-3.201166,day15,00000d53c06c43379ed22c775ce89614
9,-3.265105,day43,00000e36ecd94e5c54ab9d08c7b29163


In [26]:

# input_df.set_index(['cuid', 'day'])
input_df.index = pd.MultiIndex.from_tuples([(x, y) for x, y in zip(input_df['cuid'], input_df['day'])])

In [27]:
# Убрать дубликаты в индексе уровня 1
input_df_dr = input_df[~input_df.index.duplicated(keep='first')]
input_df.shape, input_df_dr.shape

((12874345, 3), (11446181, 3))

In [28]:
out_df = input_df_dr['pred'].unstack(level=1)

# Отсортировать индексы как задано в test
# out_df = out_df.reindex(cuid_out.index)

print(out_df.shape)
out_df.head(3)

(427994, 60)


,day1,day10,day11,day12,day13,day14,day15,day16,day17,day18,...,day55,day56,day57,day58,day59,day6,day60,day7,day8,day9
00000d2994b6df9239901389031acaac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.640258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00000d53c06c43379ed22c775ce89614,NaN,NaN,NaN,NaN,NaN,-3.146274,-3.201166,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00000e36ecd94e5c54ab9d08c7b29163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
out_df.index.nunique()

427994